In [ ]:
#觀察過資料後，因資料龐大及來不及分析文字，決定以'_score'為主軸建構模型及進行測試

In [23]:
#匯入套件
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier

In [2]:
#讀入原始檔案 data_identification.csv, emotion.csv
data_identification = pd.read_csv('data_identification.csv', sep=",")
emotion = pd.read_csv('emotion.csv', sep=",")
#分出test及train資料
test_data = data_identification[data_identification['identification'] == 'test']
train_data = data_identification[data_identification['identification'] == 'train']
#讀取並解析json檔row data
tweets = []
for line in open('tweets_DM.json', 'r'):
    tweets.append(json.loads(line))
    
tweets_df = pd.DataFrame(tweets)
tweets_df

,_score,_index,_source,_crawldate,_type
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets
2,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets
3,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets
4,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets
...,...,...,...,...,...
1867530,827,hashtag_tweets,"{'tweet': {'hashtags': ['mixedfeeling', 'butim...",2015-05-12 12:51:52,tweets
1867531,368,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x29d0...",2017-10-02 17:54:04,tweets
1867532,498,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2a6a...",2016-10-10 11:04:32,tweets
1867533,840,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x24fa...",2016-09-02 14:25:06,tweets


In [3]:
#由於資料量過於龐大，先擷取10000筆資料來建構模型
#解析'_source'欄位之json格式
tweets10000 = tweets_df.iloc[0:10000]
tmpSource10000 = tweets10000['_source']

lSource10000 = pd.DataFrame()
for i in range(0, tmpSource10000.count()):
    data = json.dumps(tmpSource10000[i])
    tmp = pd.DataFrame(json.loads(data))
    lSource10000 = lSource10000.append(tmp.T.reset_index())

lSource10000 = lSource10000.reset_index()
del lSource10000["level_0"]
lSource10000

,index,hashtags,text,tweet_id
0,tweet,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",0x376b20
1,tweet,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",0x2d5350
2,tweet,[bibleverse],"Confident of your obedience, I write to you, k...",0x28b412
3,tweet,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0
4,tweet,[],"""Trust is not the same as faith. A friend is s...",0x2de201
...,...,...,...,...
9995,tweet,[],@BlackApril91107 @DonCheadle He's pulling a Ba...,0x33938d
9996,tweet,[],Now it's time to observe and wait wisely for t...,0x331a93
9997,tweet,[],@FloydMayweather Yes!! Because your President ...,0x320983
9998,tweet,"[brighttalk, contentstrategy]",Thx @tonimantych @ #brighttalk Virtual Summit:...,0x24fb3b


In [4]:
#10000筆資料並合併解析出的'_source'
tweets10000 = tweets10000.merge(lSource10000, left_index=True, right_index=True)
tweets10000

,_score,_index,_source,_crawldate,_type,index,hashtags,text,tweet_id
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets,tweet,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",0x376b20
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets,tweet,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",0x2d5350
2,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets,tweet,[bibleverse],"Confident of your obedience, I write to you, k...",0x28b412
3,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets,tweet,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0
4,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets,tweet,[],"""Trust is not the same as faith. A friend is s...",0x2de201
...,...,...,...,...,...,...,...,...,...
9995,958,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x3393...",2015-06-19 06:13:57,tweets,tweet,[],@BlackApril91107 @DonCheadle He's pulling a Ba...,0x33938d
9996,112,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x331a...",2016-11-10 16:26:31,tweets,tweet,[],Now it's time to observe and wait wisely for t...,0x331a93
9997,847,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x3209...",2015-01-19 14:24:22,tweets,tweet,[],@FloydMayweather Yes!! Because your President ...,0x320983
9998,243,hashtag_tweets,"{'tweet': {'hashtags': ['brighttalk', 'content...",2017-09-24 16:35:51,tweets,tweet,"[brighttalk, contentstrategy]",Thx @tonimantych @ #brighttalk Virtual Summit:...,0x24fb3b


In [5]:
#移除經判斷後不必要之欄位
del tweets10000["_index"]
del tweets10000["_source"]
del tweets10000["_crawldate"]
del tweets10000["_type"]
del tweets10000["index"]

tweets10000

,_score,hashtags,text,tweet_id
0,391,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",0x376b20
1,433,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",0x2d5350
2,232,[bibleverse],"Confident of your obedience, I write to you, k...",0x28b412
3,376,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0
4,989,[],"""Trust is not the same as faith. A friend is s...",0x2de201
...,...,...,...,...
9995,958,[],@BlackApril91107 @DonCheadle He's pulling a Ba...,0x33938d
9996,112,[],Now it's time to observe and wait wisely for t...,0x331a93
9997,847,[],@FloydMayweather Yes!! Because your President ...,0x320983
9998,243,"[brighttalk, contentstrategy]",Thx @tonimantych @ #brighttalk Virtual Summit:...,0x24fb3b


In [14]:
#合併對應'tweet_id'之'emotion'
tweets10000['emotion'] = ""

for i in range(0, tweets10000['tweet_id'].count()):
    tmp = emotion.loc[emotion['tweet_id'].str.contains(tweets10000['tweet_id'][i], case=False)]
    if tmp['emotion'].count() > 0:
        tweets10000['emotion'][i] = tmp.iat[0,1]
    else:
        tweets10000['emotion'][i] = "surprise"
        
tweets10000

<ipython-input-14-91083b50c410>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets10000['emotion'][i] = tmp.iat[0,1]
<ipython-input-14-91083b50c410>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets10000['emotion'][i] = "surprise"


,_score,hashtags,text,tweet_id,emotion
0,391,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",0x376b20,anticipation
1,433,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",0x2d5350,sadness
2,232,[bibleverse],"Confident of your obedience, I write to you, k...",0x28b412,surprise
3,376,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0,fear
4,989,[],"""Trust is not the same as faith. A friend is s...",0x2de201,surprise
...,...,...,...,...,...
9995,958,[],@BlackApril91107 @DonCheadle He's pulling a Ba...,0x33938d,surprise
9996,112,[],Now it's time to observe and wait wisely for t...,0x331a93,trust
9997,847,[],@FloydMayweather Yes!! Because your President ...,0x320983,sadness
9998,243,"[brighttalk, contentstrategy]",Thx @tonimantych @ #brighttalk Virtual Summit:...,0x24fb3b,joy


In [15]:
#暫存檔案
tweets10000.to_csv("temp.csv", encoding='utf-8', index=False)

In [60]:
#暫存檔案
tweets_df.to_csv("ALLtemp.csv", encoding='utf-8', index=False)

In [34]:
#以10000筆資料區分訓練用資料
xhead10000 = ['_score']

x_train_10000 = tweets10000[xhead10000]
x_train_10000
y_train_10000 = tweets10000.emotion
y_train_10000

0       anticipation
1            sadness
2           surprise
3               fear
4           surprise
            ...     
9995        surprise
9996           trust
9997         sadness
9998             joy
9999        surprise
Name: emotion, Length: 10000, dtype: object

In [35]:
#使用DecisionTreeClassifier建立模型，進行訓練
model10000 = DecisionTreeClassifier()
model10000.fit(x_train_10000, y_train_10000)

DecisionTreeClassifier()

In [41]:
#確認訓練之結果準確度
model10000.predict(x_train_10000)

array(['sadness', 'surprise', 'anticipation', ..., 'sadness', 'joy',
       'surprise'], dtype=object)

In [75]:
#讀取測試資料對應的'id', '_score'
test = pd.read_excel('sampleSubmission總共是411973列.xlsx')

In [76]:
test

,_score,id
0,232,0x28b412
1,989,0x2de201
2,66,0x218443
3,104,0x2939d5
4,310,0x26289a
...,...,...
411967,602,0x2913b4
411968,598,0x2a980e
411969,827,0x316b80
411970,368,0x29d0cb


In [81]:
#預測後得到結果
x_test = pd.DataFrame(test['_score'])

result = model10000.predict(x_test)
result

array(['anticipation', 'joy', 'anticipation', ..., 'joy', 'surprise',
       'disgust'], dtype=object)

In [84]:
#合併結果
out = pd.DataFrame(test['id'])
out['emotion'] = result
out

,id,emotion
0,0x28b412,anticipation
1,0x2de201,joy
2,0x218443,anticipation
3,0x2939d5,joy
4,0x26289a,anticipation
...,...,...
411967,0x2913b4,surprise
411968,0x2a980e,surprise
411969,0x316b80,joy
411970,0x29d0cb,surprise


In [85]:
#輸出成csv
out.to_csv("result.csv", encoding='utf-8', index=False)

In [98]:
train_data1 = pd.read_excel('ALLtemp1_Emotion_468117.xlsx')
train_data2 = pd.read_excel('ALLtemp2_Emotion_466955.xlsx')
#train_data = pd.merge(train_data1, train_data2, left_index='True', right_index='True')
#train_data
train_data = pd.concat([train_data1,train_data2])
train_data = train_data.reset_index()
train_data

,index,_score,id,emotion
0,0,1021,0x2c91a8,anticipation
1,1,481,0x368e95,joy
2,2,631,0x359db9,anticipation
3,3,560,0x1fde89,anger
4,4,728,0x213b7f,anticipation
...,...,...,...,...
935067,466950,81,0x26386e,disgust
935068,466951,25,0x2ee290,trust
935069,466952,94,0x1f1592,sadness
935070,466953,798,0x31bf05,trust


In [99]:
x_train = pd.DataFrame(train_data['_score'])
y_train = pd.DataFrame(train_data['emotion'])
model = DecisionTreeClassifier()
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [100]:
x_test

,_score
0,232
1,989
2,66
3,104
4,310
...,...
411967,602
411968,598
411969,827
411970,368


In [101]:
result = model.predict(x_test)
result

array(['joy', 'joy', 'joy', ..., 'joy', 'joy', 'joy'], dtype=object)

In [104]:
out = pd.DataFrame(test['id'])
out['emotion'] = result
out

,id,emotion
0,0x28b412,joy
1,0x2de201,joy
2,0x218443,joy
3,0x2939d5,joy
4,0x26289a,joy
...,...,...
411967,0x2913b4,joy
411968,0x2a980e,joy
411969,0x316b80,joy
411970,0x29d0cb,joy


In [105]:
out.to_csv("result1.csv", encoding='utf-8', index=False)

In [ ]:
#檔案資料量過大，拆分3個檔案來處理
emotion
temp1 = emotion.iloc[0:500000]
temp2 = emotion.iloc[500000:1000000]
temp3 = emotion.iloc[1000000:emotion['emotion'].count()]

temp1.to_csv("emotion1.csv", encoding='utf-8', index=False)
temp2.to_csv("emotion2.csv", encoding='utf-8', index=False)
temp3.to_csv("emotion3.csv", encoding='utf-8', index=False)

In [67]:
#檔案資料量過大，拆分3個檔案來處理
temp1 = tweets_df.iloc[0:600000]
temp2 = tweets_df.iloc[600000:1200000]
temp3 = tweets_df.iloc[1200000:tweets_df['_score'].count()]

temp1.to_csv("ALLtemp1.csv", encoding='utf-8', index=False)
temp2.to_csv("ALLtemp2.csv", encoding='utf-8', index=False)
temp3.to_csv("ALLtemp3.csv", encoding='utf-8', index=False)